In [1]:
from quarto import objects, players
import logging
import math
from copy import deepcopy

In [8]:
GAMES = 1000

scores = {
    'me': 0,
    'opponent': 0,
    'draw': 0
}

home = {
    0: 'me',
    1: 'opponent',
    -1: 'draw'
}

away = {
    0: 'opponent',
    1: 'me',
    -1: 'draw'
}

for _ in range(GAMES // 2):
    game = objects.Quarto()
    game.set_players((players.MinMaxPlayer(game, 1), players.RandomPlayer(game)))
    winner = home[game.run()]
    scores[winner] += 1

for _ in range(GAMES // 2):
    game = objects.Quarto()
    game.set_players((players.RandomPlayer(game), players.MinMaxPlayer(game, 1)))
    winner = away[game.run()]
    scores[winner] += 1

logging.warning(f"Stats: {scores}") 

In [43]:
def minmax(max_depth=math.inf, print_stats = False):
    max_depth_reached = 0
    cache = {}
    hits = 0
    misses = 0
    MAX_CACHE_LEN = 1e6

    def minmax_with_pruning(state: objects.Quarto, alpha, beta, depth=0):

        nonlocal max_depth_reached, cache, hits, misses
        max_depth_reached = max(max_depth_reached, depth)

        winner = state.check_winner()  
        if state.check_finished() or winner != -1:
            loser = 1 - 2 * winner
            return loser, None 

        if depth >= max_depth:
            return 0, None        
        
        available_pieces = list(i for i in range(state.BOARD_SIDE ** 2) if i not in state.get_board_status() and i != state.get_selected_piece())
        available_positions = list((j // 4, j % 4) for j in range(state.BOARD_SIDE ** 2) if state.get_board_status()[j // 4][j % 4] < 0)

        #logging.warning(f'Depth: {depth} Player: {state.get_current_player()} Pieces: {available_pieces} Positions: {available_positions} Board: {state.get_board_status()}')

        if state.get_current_player() == 0: # my turn, minimize
            value = (math.inf, None)
            for x, y in available_positions:
                tmp = deepcopy(state)

                if tmp.get_selected_piece() not in tmp.get_board_status() and tmp.get_selected_piece() != -1:
                    r = tmp.place(y, x)

                for p in available_pieces:
                    tmp2 = deepcopy(tmp)
                    tmp2.select(p)
                    tmp2.end_player_turn()
                    val, _ = minmax_with_pruning(tmp2, alpha, beta, depth + 1)

                    # logging.warning(f'Move: {(val, ((x, y), p))} by player at depth {depth}')
                    
                    value = min(value, (val, ((x, y), p)))

                    if value <= alpha:
                        break
                        # return value  # To check if it is okay to break two cycles at once

                    beta = min(beta, value)   
            return value
        else: # its turn, maximize
            value = (-math.inf, None)
            for x, y in available_positions:
                tmp = deepcopy(state)

                if tmp.get_selected_piece() not in tmp.get_board_status() and tmp.get_selected_piece() != -1:
                    r = tmp.place(y, x)
                    
                for p in available_pieces:
                    tmp2 = deepcopy(tmp)
                    tmp2.select(p)
                    tmp2.end_player_turn()
                    val, _ = minmax_with_pruning(tmp2, alpha, beta, depth + 1)

                    # logging.warning(f'Move: {(val, ((x, y), p))} by opponent at depth {depth}')
                
                    value = max(value, (val, ((x, y), p)))
                    
                    if value >= beta:
                        break
                        # return value  # To check if it is okay to break two cycles at once

                    alpha = max(alpha, value)    
            return value

    def minmax_strategy_with_pruning(game: objects.Quarto):
        _, move = minmax_with_pruning(game, (-math.inf, None), (math.inf, None), 0)
        nonlocal max_depth_reached, hits, misses
        
        if print_stats:  
            logging.warning(f'Chosen situation: {(_, move)}, Max depth reached: {max_depth_reached}')      
            #logging.info(f'Max depth reached: {max_depth_reached}, Cache hit ratio: {round(hits/(hits+misses), 3)*100}% ({hits}/{hits+misses}), Cache entries : {len(cache)}')

        return move

    return minmax_strategy_with_pruning

In [45]:
game = objects.Quarto([[0, 15, 10, -1], [13, 11, -1,  9], [8, -1,  6,  5], [14,  2,  7,  1]])
strategy = minmax(3)
strategy(game)

((0, 3), 3)